# Solutions to the exercises from the [Dask notebook](0_Dask.ipynb)

## Exercise: Run plane segmentation in parallel using ``dask.delayed`` 

Create a pseudo Z-stack of dimension (10, 100, 100) (ZYX)

In [10]:
import dask.array as da
planes = da.random.random((10, 100, 100), chunks=(2, 10, 10))
print(planes.shape)

(10, 100, 100)


If you are running the notebook in [Google Colab](https://colab.research.google.com/), run the cell below to install required dependencies. 

In [ ]:
%pip install dask-image fsspec>=0.3.3

Create a function ``analyze`` taking the z-section as a parameter.
Return the label_image and the z-section so we can identify when we parse the results.

In [11]:
import dask
import dask_image.ndfilters
import dask_image.ndmeasure

def analyze(z):
    plane = planes[z, :, :]
    smoothed_image = dask_image.ndfilters.gaussian_filter(plane, sigma=[1, 1])
    threshold_value = 0.33 * da.max(smoothed_image).compute()
    threshold_image = smoothed_image > threshold_value
    label_image, num_labels = dask_image.ndmeasure.label(threshold_image)
    name = "z:%s" % (z)
    return label_image, name

Segment every plane across the z-stack:
 * First we prepare the graph
 * The we compute

In [12]:
from dask import delayed

lazy_results = []
for z in range(planes.shape[0]):
    r = delayed(analyze)(z)
    lazy_results.append(r)

In [13]:
%time results = dask.compute(*lazy_results)

CPU times: user 6.41 s, sys: 427 ms, total: 6.84 s
Wall time: 6.58 s


View the results

In [15]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *

def display(i=0):
    r, name = results[i]
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(r)
    plt.title(name)
    fig.canvas.flush_events()

interact(display, i= widgets.IntSlider(value=0, min=0, max=len(results)-1, step=1, description="Select Plane", continuous_update=False))


interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Plane', max=9), Output()…

<function __main__.display(i=0)>

## Exercise: Run plane segmentation in parallel using a ``Cluster`` 

Note that this section will **not** work in Google Colab.

Create a pseudo Z-stack of dimension (10, 100, 100) (ZYX)

In [16]:
planes = da.random.random((10, 100, 100), chunks=(2, 10, 10))
print(planes.shape)

(10, 100, 100)


Create a function ``analyze`` taking the z-section as a parameter.
Return the label_image and the z-section so we can identify when we parse the results.

In [ ]:
def analyze(z):
    plane = planes[z, :, :]
    smoothed_image = dask_image.ndfilters.gaussian_filter(plane, sigma=[1, 1])
    threshold_value = 0.33 * da.max(smoothed_image).compute()
    threshold_image = smoothed_image > threshold_value
    label_image, num_labels = dask_image.ndmeasure.label(threshold_image)
    name = "z:%s" % (z)
    return label_image, name

Prepare the call

In [ ]:
def prepare_call(client, dim_z):
    futures = []
    for z in range(dim_z):
        futures.append(client.submit(analyze, z))
    return futures

Create a default local cluster

In [17]:
from dask.distributed import Client, LocalCluster

In [ ]:
cluster = LocalCluster()

with Client(cluster) as client:
    # perform code
    futures = prepare_call(clien, planes.shape[0])
    results = client.gather(futures)